## Aprendizado de Máquina Federado
### Teste local do tensorflow_federated com simulação local de dispositivos de borda

In [66]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [67]:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [68]:
# 1. Carregar o dataset Federado
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

# 2. Definir as funções de pré-processamento
NUM_CLIENTS = 10
BATCH_SIZE = 20
PREFETCH_BUFFER = 10

def preprocess(dataset):
    """Pré-processa o dataset para o modelo Keras."""

    # Mapear imagens para floats de 0-1 e redimensionar para vetores de 784
    def batch_format_fn(element):
        return (tf.reshape(element['pixels'], [-1, 784]),  # Imagens (28*28 = 784)
                tf.reshape(element['label'], [-1, 1]))    # Rótulos

    return dataset.batch(BATCH_SIZE).map(batch_format_fn).repeat(5).prefetch(PREFETCH_BUFFER)

# 3. Aplicar o pré-processamento nos dados de treinamento
# Vamos selecionar apenas um subconjunto de clientes para a simulação
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x)) 
                        for x in sample_clients]

In [80]:
# 1. Carregar o dataset Federado
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

# 2. Definir as funções de pré-processamento
NUM_CLIENTS = 10
BATCH_SIZE = 5
PREFETCH_BUFFER = 10

def preprocess(dataset):
    """Pré-processa o dataset para o modelo Keras."""
    
    # Mapear imagens para floats de 0-1 e redimensionar (e normalizar, se a normalização não estiver no pipeline de dados original)
    def batch_format_fn(element):
        # Normalização dos pixels e Reshape 28*28 -> 784
        pixels = tf.cast(element['pixels'], tf.float32) / 255.0 
        
        # Corrigido: Rótulo deve ser 1D para SparseCategoricalCrossentropy
        return (tf.reshape(pixels, [-1, 784]), 
                tf.reshape(element['label'], [-1])) 

    # Aumentando o repeat para 5 para acelerar a convergência (E=5)
    return dataset.batch(BATCH_SIZE).map(batch_format_fn).repeat(5).prefetch(10)

# 3. Aplicar o pré-processamento (Restante do seu código)
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x)) 
                        for x in sample_clients]

In [81]:
def create_keras_model():
    """Cria um modelo Keras simples para classificação MNIST."""
    # O input é um vetor de 784 (28x28) pixels
    initializer = tf.keras.initializers.GlorotNormal(seed=1)
    
#    return tf.keras.models.Sequential([
#        tf.keras.layers.InputLayer(input_shape=(784,)),
#        tf.keras.layers.Dense(10, kernel_initializer=initializer), # 10 classes de saída (0-9)
#        tf.keras.layers.Softmax(),
#    ])

    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(784,)),
        # Camada de processamento oculta
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer=initializer),
        # Camada de saída
        tf.keras.layers.Dense(10, kernel_initializer=initializer),
        tf.keras.layers.Softmax(),
    ])

def model_fn():
    """Compila o modelo Keras e o empacota para TFF."""
    keras_model = create_keras_model()
    
    # O TFF requer um modelo Keras com um Optimizer para treinar localmente
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
# Construir o processo FedAvg
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    # Otimizador usado por CADA cliente para treinamento local
    client_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=0.001),
    # Otimizador usado pelo SERVIDOR para agregar os pesos
    server_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=0.1) 
)

# Inicializar o estado global do modelo
state = iterative_process.initialize()
print("Modelo Global Inicializado.")

Modelo Global Inicializado.


In [83]:
NUM_ROUNDS = 100
print(f"Iniciando Simulação com {NUM_ROUNDS} Rodadas...")

for round_num in range(1, NUM_ROUNDS + 1):
    # Executar a próxima rodada de treinamento federado
    state, metrics = iterative_process.next(state, federated_train_data)
    
    # Exibir as métricas de desempenho agregadas no servidor
    train_metrics = metrics['client_work']['train']
    
    print(f"\n--- Rodada {round_num:2d} ---")
    print(f"  Perda (Loss) agregada: {train_metrics['loss']:.4f}")
    print(f"  Acurácia agregada: {train_metrics['sparse_categorical_accuracy']:.4f}")

Iniciando Simulação com 100 Rodadas...


2025-12-03 12:00:32.059511: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-03 12:00:32.059692: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-12-03 12:00:32.117059: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-03 12:00:32.117267: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-12-03 12:00:32.123246: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-03 12:00:32.123409: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-12-03 12:00:32.131339: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-03 12:00:32.131503: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session



--- Rodada  1 ---
  Perda (Loss) agregada: 2.3028
  Acurácia agregada: 0.0907

--- Rodada  2 ---
  Perda (Loss) agregada: 2.3028
  Acurácia agregada: 0.0918

--- Rodada  3 ---
  Perda (Loss) agregada: 2.3028
  Acurácia agregada: 0.0914

--- Rodada  4 ---
  Perda (Loss) agregada: 2.3027
  Acurácia agregada: 0.0949

--- Rodada  5 ---
  Perda (Loss) agregada: 2.3027
  Acurácia agregada: 0.0973

--- Rodada  6 ---
  Perda (Loss) agregada: 2.3027
  Acurácia agregada: 0.0988

--- Rodada  7 ---
  Perda (Loss) agregada: 2.3027
  Acurácia agregada: 0.1002

--- Rodada  8 ---
  Perda (Loss) agregada: 2.3027
  Acurácia agregada: 0.0998

--- Rodada  9 ---
  Perda (Loss) agregada: 2.3026
  Acurácia agregada: 0.1006

--- Rodada 10 ---
  Perda (Loss) agregada: 2.3026
  Acurácia agregada: 0.1021

--- Rodada 11 ---
  Perda (Loss) agregada: 2.3026
  Acurácia agregada: 0.1025

--- Rodada 12 ---
  Perda (Loss) agregada: 2.3026
  Acurácia agregada: 0.1014

--- Rodada 13 ---
  Perda (Loss) agregada: 2.3026
 